# Data Modeling with Cassandra

## Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Part II.  Apache Cassandra Database for Query. 

We will work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# make a connection to a Cassandra instance 

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS raywong 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('raywong')
except Exception as e:
    print(e)

#### Task1: Get the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.

For this task, we will set the sessionId and the itemInSession to be the PRIMARY KEY because we are supposed to identify the rows meet the certain sessionId value and itemInSession value.

In [8]:
# load the csv file into a pandas dataframe
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73
2,Morcheeba,Jacob,M,1,Klein,257.41016,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Women Lose Weight (Feat: Slick Rick),73
3,Maroon 5,Jacob,M,2,Klein,231.23546,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Won't Go Home Without You,73
4,Train,Jacob,M,3,Klein,216.76363,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Hey_ Soul Sister,73


In [9]:
# get columns
df.columns

Index(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
       'level', 'location', 'sessionId', 'song', 'userId'],
      dtype='object')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
artist           6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
sessionId        6820 non-null int64
song             6820 non-null object
userId           6820 non-null int64
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [11]:
# get index of each column
[(c, df.columns.get_loc(c))for c in df.columns]

[('artist', 0),
 ('firstName', 1),
 ('gender', 2),
 ('itemInSession', 3),
 ('lastName', 4),
 ('length', 5),
 ('level', 6),
 ('location', 7),
 ('sessionId', 8),
 ('song', 9),
 ('userId', 10)]

In [12]:
# create artist_song table for task1
query = 'CREATE TABLE IF NOT EXISTS artist_song (sessionId int, itemInSession int, artist text, song text, \
            length float, PRIMARY KEY (sessionId, itemInSession))'
try:
    session.execute(query)
except Exception as e:
    print(e)               

In [13]:
# insert values into artist_song table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO artist_song (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [14]:
# write SELECT statement to return the result
query = "SELECT artist, song, length FROM artist_song WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


#### Task2: Get the name of artist, song (sorted by itemInSession) and user (first and last name) for userId = 10, sessionId = 182

For this tasks we will set userId and sessionId to be PRIMARY KEY.

In [15]:
# create artist_song_user table and insert values into it

query = 'CREATE TABLE IF NOT EXISTS artist_song_user (userId int, sessionId int, itemInSession int, artist text, song text, \
            firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))'
try:
    session.execute(query)
except Exception as e:
    print(e)     

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO artist_song_user (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0],line[9], line[1], line[4]))
                    

In [16]:
# SELECT statement 
query = "SELECT artist, song, firstName, lastName FROM artist_song_user WHERE userId=10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print(row)
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### Task3: Get every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'

To get the user's first name and last name, we need the userId to be a PRIMARY KEY to identify the user. Also we need to identify the song 'All Hands Against His Own', so song should be another PRIMARY KEY. 

The sequence of these two PRIMARY KEYS is little tricky here. We need to put song before userId, since we will only use song in the WHERE clause, userId will not be used in the WHERE clause.

In [17]:
# create song_user table

query = 'CREATE TABLE IF NOT EXISTS song_user (song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))'
try:
    session.execute(query)
except Exception as e:
    print(e)     

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_user (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
                    

In [18]:
## SELECT statement 

query = "SELECT firstName, lastName FROM song_user WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print(row)
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [19]:
## Drop the table before closing out the sessions
query = "DROP TABLE artist_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE artist_song_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE song_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [20]:
session.shutdown()
cluster.shutdown()